In [ ]:
import os
import datetime
from tqdm import tqdm
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from championWrapper import ChampionWrapper

In [ ]:
in_data = pd.read_csv('pb_table.csv', index_col=False)
champs = ChampionWrapper()

num_champs = len(champs.champToID)
num_champs


In [ ]:
#isolate the input frames of hero choices by name
isolated_input = in_data[['BB1', 'RB1', 'BB2', 'RB2', 'BB3', 'RB3', 'BP1', 'RP1','RP2','BP2','BP3', 'RP3', 'RB4','BB4', 'RB5', 'BB5', 'RP4', 'BP4','BP5', 'RP5']]
isolated_input.head()

In [ ]:
#Dataframe to Numpy
inputnpArray = isolated_input.to_numpy()
inputnpArray


In [ ]:
# Numpy convert Names to Ids
for match in range(len(inputnpArray)):
    for column in range(len(inputnpArray[match])):
        inputnpArray[match][column] = champs.cNameToId(inputnpArray[match][column])

inputnpArray



Here we have the data   
In Raw form we want to make it into [In][Label] sets to    
use in our Sequence to Sequence model   

In [ ]:
#Here we have the data
# In Raw form we want to make it into [In][Label] sets to 
# use in our Sequence to Sequence model

In [ ]:
numberpd = pd.DataFrame(inputnpArray, columns=['BB1', 'RB1', 'BB2', 'RB2', 'BB3', 'RB3', 'BP1', 'RP1','RP2','BP2','BP3', 'RP3', 'RB4','BB4', 'RB5', 'BB5', 'RP4', 'BP4','BP5', 'RP5'])
print(numberpd)

In [ ]:
traindf, testdf = train_test_split(numberpd, test_size=0.2)
testdf, valdf = train_test_split(testdf, test_size=0.5)

In [ ]:
traindf.head()

In [ ]:
testdf.head()


In [ ]:
valdf.head()

In [ ]:
traindf.shape

In [ ]:
testdf.shape

In [ ]:
valdf.shape

In [ ]:
from WindowGenerator import WindowGenerator

wg = WindowGenerator(9, 1, 1, train_df=traindf, val_df=valdf, test_df=testdf, label_columns=None)
wg

In [ ]:
tfl = tf.keras.layers
n_features=num_champs
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15)
from WindowGenerator import SplitGen

splitter = SplitGen()
train, test = train_test_split(inputnpArray, test_size=0.2)
test, val = train_test_split(test, test_size=0.5)

i=19
o=1
f=num_champs

X_train, y_train = splitter.split_sequences(train, i, o, f)
X_test, y_test = splitter.split_sequences(test, i, o, f)
X_val, y_val = splitter.split_sequences(val, i, o, f)


In [27]:
doubleLSTM_model = tf.keras.Sequential([
    tfl.LSTM(512, activation='relu', input_shape=(i, f)),
    tfl.RepeatVector(o),
    tfl.LSTM(256, activation='relu', return_sequences=True),
    tfl.TimeDistributed(tfl.Dense(158))
])
doubleLSTM_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

doubleLSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=200)
print(doubleLSTM_model.evaluate(X_test, y_test))


Epoch 1/200
102/102 [==============================] - 8s 69ms/step - loss: 0.0062 - accuracy: 0.0365 - val_loss: 0.0062 - val_accuracy: 0.0490
Epoch 2/200
102/102 [==============================] - 6s 63ms/step - loss: 0.0062 - accuracy: 0.0521 - val_loss: 0.0062 - val_accuracy: 0.0686
Epoch 3/200
102/102 [==============================] - 7s 67ms/step - loss: 0.0062 - accuracy: 0.0622 - val_loss: 0.0062 - val_accuracy: 0.0784
Epoch 4/200
102/102 [==============================] - 7s 65ms/step - loss: 0.0062 - accuracy: 0.0923 - val_loss: 0.0062 - val_accuracy: 0.0735
Epoch 5/200
 32/102 [========>.....................] - ETA: 4s - loss: 0.0061 - accuracy: 0.1104